In [1]:
Output = ('/Users/alexis/Library/CloudStorage/OneDrive-UniversityofNorthCarolinaatChapelHill/CEMALB_DataAnalysisPM/Projects/P1011. NC Well Arsenic/P1011.2. Analyses/P1011.2.2. Arsenic Prediction/Output')
cur_date = "012224"

library(readxl)
library(openxlsx)
library(lubridate)
library(tidyverse)
library(gtsummary)
library(caret)
library(e1071)
library(Hmisc)
library(randomForest)
library(pROC)
library(themis)
#library(rlang)

# reading in file
well_data = data.frame(read_excel("Input/Imputed_Well_Data_080823.xlsx")) 


Attaching package: ‘lubridate’


The following objects are masked from ‘package:base’:

    date, intersect, setdiff, union


── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr   1.1.3     ✔ readr   2.1.4
✔ forcats 1.0.0     ✔ stringr 1.5.0
✔ ggplot2 3.4.3     ✔ tibble  3.2.1
✔ purrr   1.0.2     ✔ tidyr   1.3.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
Loading required package: lattice


Attaching package: ‘caret’


The following object is masked from ‘package:purrr’:

    lift



Attaching package: ‘Hmisc’


The following object is masked from ‘package:e1071’:

    impute


The following objects are masked from ‘package:dplyr’:

    src, summarize


The following objects are masked from ‘package:base’:

    format.pval, units


rand

In [2]:
head(well_data)

,Tax_ID,Health_Dept_ID,Permit_No,Water_Sample_Date,Casing_Depth,Well_Depth,Static_Water_Depth,Flow_Rate,pH,Geology,Soil_Type,Landuse,Longitude,Latitude,Stream_Distance,Elevation,Metal,Concentration,Detect_Concentration
,<chr>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<chr>
1,1006004,960,12-155,9/24/12,52,165,41,60.0,7.7,Carolina Slate Belt - CZmd3,BdB2,81,-80.29918,35.17970,811.9613,611.499,Ar,1.531414,ND
2,1024009,1803,15-177,12/17/15,40,445,42,2.0,7.3,Carolina Slate Belt - CZfv2,GsB,81,-80.31061,35.15487,341.7654,688.452,Ar,2.348528,ND
3,1054019,1647,14-212,2/2/15,45,160,40,40.0,7.4,Carolina Slate Belt - CZmd3,GsB,81,-80.33170,35.16158,634.2669,692.789,Ar,2.043848,ND
4,1057017,970,12-169,10/22/12,42,440,57,1.5,8.0,Carolina Slate Belt - CZmd2,BaB,42,-80.32263,35.13962,855.4713,692.789,Ar,35.000000,D
5,1060006,533,10-239,1/3/11,48,120,42,25.0,7.1,Carolina Slate Belt - CZmd2,BdB2,43,-80.32911,35.13027,163.5688,683.387,Ar,2.297118,ND
6,1066006,1795,15-224,12/15/15,60,280,32,10.0,8.2,Carolina Slate Belt - CZmd2,BaB,22,-80.32205,35.10479,725.7672,614.599,Ar,53.000000,D


Using `water_sample_date`, `casing_depth`, `well_depth`, `static_water_depth`, `pH`, `flow_rate`, `geology`, `soil_type`, `landuse`, `latitude`, and `longitude` to predict with As concentration falls above or below a certain threshold. RF and SVM?? models will be built to predict concentration that has binarized in 3 different ways:

1. Detect vs. Non-Detect
2. Limit of Detection (< or >= 5ppb)
3. EPA's Maximum Contaminant Level (MCL) (< or >= 10ppb)

Starting by creating 2 additional variables for above and below 5 and 10ppb and calculating some summary statistics to determine if there are any signficiant differenes between the predictor variables for each outcome variable, but omitted categorical variables for simplicity. 

In [3]:
arsenic_data = well_data %>%
    mutate(Five_Threshold = relevel(factor(ifelse(Concentration >= 5, 1, 0)), ref = "0"),
          Ten_Threshold = relevel(factor(ifelse(Concentration >= 10, 1, 0)), ref = "0"),
           # making this col into a factor
          Detect_Concentration = relevel(factor(ifelse(Detect_Concentration == "D", 1, 0)), ref = "0"),
          # converting water sample date from a character to a date type 
          Water_Sample_Date = mdy(Water_Sample_Date),
          # converting missing records to an "other" category
          Landuse = ifelse(Landuse %in% NA, "Other", Landuse)) %>%

    # filtering for Arsenic only
    filter(Metal == "Ar") %>%
    # cleaning up geological categories 
    separate(Geology, c(NA, "Geology"), sep = "-") 

head(arsenic_data)

Warning message:
“There was 1 warning in `mutate()`.
ℹ In argument: `Water_Sample_Date = mdy(Water_Sample_Date)`.
Caused by warning:
!  6 failed to parse.”


,Tax_ID,Health_Dept_ID,Permit_No,Water_Sample_Date,Casing_Depth,Well_Depth,Static_Water_Depth,Flow_Rate,pH,Geology,⋯,Landuse,Longitude,Latitude,Stream_Distance,Elevation,Metal,Concentration,Detect_Concentration,Five_Threshold,Ten_Threshold
,<chr>,<dbl>,<chr>,<date>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,⋯,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<fct>,<fct>,<fct>
1,1006004,960,12-155,2012-09-24,52,165,41,60.0,7.7,CZmd3,⋯,81,-80.29918,35.17970,811.9613,611.499,Ar,1.531414,0,0,0
2,1024009,1803,15-177,2015-12-17,40,445,42,2.0,7.3,CZfv2,⋯,81,-80.31061,35.15487,341.7654,688.452,Ar,2.348528,0,0,0
3,1054019,1647,14-212,2015-02-02,45,160,40,40.0,7.4,CZmd3,⋯,81,-80.33170,35.16158,634.2669,692.789,Ar,2.043848,0,0,0
4,1057017,970,12-169,2012-10-22,42,440,57,1.5,8.0,CZmd2,⋯,42,-80.32263,35.13962,855.4713,692.789,Ar,35.000000,1,1,1
5,1060006,533,10-239,2011-01-03,48,120,42,25.0,7.1,CZmd2,⋯,43,-80.32911,35.13027,163.5688,683.387,Ar,2.297118,0,0,0
6,1066006,1795,15-224,2015-12-15,60,280,32,10.0,8.2,CZmd2,⋯,22,-80.32205,35.10479,725.7672,614.599,Ar,53.000000,1,1,1


In [4]:
# original number of records
dim(well_data)

# records kept for analysis
dim(arsenic_data)

[1] 2145   19

[1] 715  21

# Variable Selection

To determine which variables are collinear and need to be removed, but this can only be done with the continuous data. 

In [5]:
# creating correlation matrix
corr_matrix = rcorr(as.matrix(arsenic_data[c(5:9,13:16)]), type = "spearman")
# creating df where I can easily view coefficents that are highly correlated (rho > 0.7 or rho < -0.7)
corr_matrix_df = data.frame(corr_matrix$r) %>%
  rownames_to_column(var = "Variable1") %>%
  # needed to specificy package of melt for it to work
  reshape2::melt(variable = "Variable2", value.name = "Spearman Coefficient") %>%
  # also removed values = 1, because those were correlations between the same variable
  filter(abs(`Spearman Coefficient`) < 1 & abs(`Spearman Coefficient`) > 0.7) %>%
  arrange(desc(`Spearman Coefficient`)) %>%
  # removing duplicate spearman coefficient values and keeping all columns
  distinct(`Spearman Coefficient`, .keep_all = TRUE) 

head(corr_matrix_df)

Using Variable1 as id variables



Variable1,Variable2,Spearman Coefficient
<chr>,<fct>,<dbl>


No features passed the threshold, so they aren't highly correlated to each other.

In [6]:
# summary statistics df: detect_concentration
arsenic_data %>%
  tbl_summary(by = Detect_Concentration, missing = "no", 
  include = colnames(arsenic_data[c(4:16)]), 
              statistic = list(all_continuous() ~ "{mean} ({sd})")) %>%
  add_n() %>% 
  add_p(test = list(all_continuous() ~ "aov")) %>% # adding p value from anova
  as_tibble()

There was an error in 'add_p()/add_difference()' for variable 'Geology', p-value omitted:
Error in stats::fisher.test(c(" CZmd3", " CZfv2", " CZmd3", " CZmd2", : FEXACT error 7(location). LDSTP=18510 is too small for this problem,
  (pastp=33.69, ipn_0:=ipoin[itp=440]=17951, stp[ipn_0]=17.2264).
Increase workspace or consider using 'simulate.p.value=TRUE'

There was an error in 'add_p()/add_difference()' for variable 'Soil_Type', p-value omitted:
Error in stats::fisher.test(c("BdB2", "GsB", "GsB", "BaB", "BdB2", "BaB", : FEXACT error 7(location). LDSTP=18390 is too small for this problem,
  (pastp=11.3975, ipn_0:=ipoin[itp=528]=12000, stp[ipn_0]=6.49224).
Increase workspace or consider using 'simulate.p.value=TRUE'

There was an error in 'add_p()/add_difference()' for variable 'Landuse', p-value omitted:
Error in stats::fisher.test(c("81", "81", "81", "42", "43", "22", "81", : FEXACT error 7(location). LDSTP=18510 is too small for this problem,
  (pastp=25.3394, ipn_0:=ipoin[itp=437]=8

**Characteristic**,**N**,"**0**, N = 517","**1**, N = 198",**p-value**
<chr>,<chr>,<chr>,<chr>,<chr>
Water_Sample_Date,713,2013-06-05 (979.174260670888),2013-03-05 (957.843005291701),0.3
Casing_Depth,715,74 (33),55 (23),<0.001
Well_Depth,715,301 (144),334 (128),0.005
Static_Water_Depth,715,35 (12),36 (13),0.5
Flow_Rate,715,25 (33),14 (16),<0.001
pH,715,7.45 (0.54),7.82 (0.40),<0.001
Geology,715,NA,NA,NA
CZfv,NA,118 (23%),7 (3.5%),NA
CZfv2,NA,7 (1.4%),2 (1.0%),NA


In [7]:
# summary statistics df: five_threshold
arsenic_data %>%
  tbl_summary(by = Five_Threshold, missing = "no", 
  include = colnames(arsenic_data[c(4:16)]), 
              statistic = list(all_continuous() ~ "{mean} ({sd})")) %>%
  add_n() %>% 
  add_p(test = list(all_continuous() ~ "aov")) %>% # adding p value from anova
  as_tibble()

There was an error in 'add_p()/add_difference()' for variable 'Geology', p-value omitted:
Error in stats::fisher.test(c(" CZmd3", " CZfv2", " CZmd3", " CZmd2", : FEXACT error 7(location). LDSTP=18510 is too small for this problem,
  (pastp=37.197, ipn_0:=ipoin[itp=363]=18102, stp[ipn_0]=34.4626).
Increase workspace or consider using 'simulate.p.value=TRUE'

There was an error in 'add_p()/add_difference()' for variable 'Soil_Type', p-value omitted:
Error in stats::fisher.test(c("BdB2", "GsB", "GsB", "BaB", "BdB2", "BaB", : FEXACT error 7(location). LDSTP=18390 is too small for this problem,
  (pastp=16.9551, ipn_0:=ipoin[itp=126]=3667, stp[ipn_0]=1.38629).
Increase workspace or consider using 'simulate.p.value=TRUE'

There was an error in 'add_p()/add_difference()' for variable 'Landuse', p-value omitted:
Error in stats::fisher.test(c("81", "81", "81", "42", "43", "22", "81", : FEXACT error 7(location). LDSTP=18510 is too small for this problem,
  (pastp=24.934, ipn_0:=ipoin[itp=596]=30

**Characteristic**,**N**,"**0**, N = 521","**1**, N = 194",**p-value**
<chr>,<chr>,<chr>,<chr>,<chr>
Water_Sample_Date,713,2013-06-01 (979.73581125107),2013-03-12 (956.560244097347),0.3
Casing_Depth,715,74 (33),55 (23),<0.001
Well_Depth,715,300 (144),336 (129),0.003
Static_Water_Depth,715,35 (12),35 (13),0.6
Flow_Rate,715,25 (33),14 (16),<0.001
pH,715,7.45 (0.54),7.83 (0.40),<0.001
Geology,715,NA,NA,NA
CZfv,NA,118 (23%),7 (3.6%),NA
CZfv2,NA,7 (1.3%),2 (1.0%),NA


In [8]:
# summary statistics df: ten_threshold
arsenic_data %>%
  tbl_summary(by = Ten_Threshold, missing = "no", 
  include = colnames(arsenic_data[c(4:16)]), 
              statistic = list(all_continuous() ~ "{mean} ({sd})")) %>%
  add_n() %>% 
  add_p(test = list(all_continuous() ~ "aov")) %>% # adding p value from anova
  as_tibble()

There was an error in 'add_p()/add_difference()' for variable 'Geology', p-value omitted:
Error in stats::fisher.test(c(" CZmd3", " CZfv2", " CZmd3", " CZmd2", : FEXACT error 7(location). LDSTP=18510 is too small for this problem,
  (pastp=32.2004, ipn_0:=ipoin[itp=525]=18099, stp[ipn_0]=20.263).
Increase workspace or consider using 'simulate.p.value=TRUE'

There was an error in 'add_p()/add_difference()' for variable 'Soil_Type', p-value omitted:
Error in stats::fisher.test(c("BdB2", "GsB", "GsB", "BaB", "BdB2", "BaB", : FEXACT error 7(location). LDSTP=18390 is too small for this problem,
  (pastp=9.36426, ipn_0:=ipoin[itp=91]=4350, stp[ipn_0]=7.59085).
Increase workspace or consider using 'simulate.p.value=TRUE'

There was an error in 'add_p()/add_difference()' for variable 'Landuse', p-value omitted:
Error in stats::fisher.test(c("81", "81", "81", "42", "43", "22", "81", : FEXACT error 7(location). LDSTP=18510 is too small for this problem,
  (pastp=43.3001, ipn_0:=ipoin[itp=34]=5, 

**Characteristic**,**N**,"**0**, N = 583","**1**, N = 132",**p-value**
<chr>,<chr>,<chr>,<chr>,<chr>
Water_Sample_Date,713,2013-05-12 (972.978329886697),2013-05-03 (979.450720926567),>0.9
Casing_Depth,715,73 (33),51 (19),<0.001
Well_Depth,715,306 (147),327 (109),0.12
Static_Water_Depth,715,35 (13),34 (11),0.4
Flow_Rate,715,24 (32),13 (15),<0.001
pH,715,7.48 (0.54),7.87 (0.41),<0.001
Geology,715,NA,NA,NA
CZfv,NA,123 (21%),2 (1.5%),NA
CZfv2,NA,8 (1.4%),1 (0.8%),NA


Although the p values are very significant between the classes of each outcome, there is a high level of class imbalance which is likely to affect model performance. Class imbalance will be addressed using SMOTE. 

In [9]:
# creating dfs for each outcome
# dropped 2 rows that had missing dates
arsenic_detect_df = drop_na(arsenic_data[,c(4:16,19)]) 
arsenic_five_df = drop_na(arsenic_data[,c(4:16,20)]) 
arsenic_ten_df = drop_na(arsenic_data[,c(4:16,21)]) 

head(arsenic_detect_df)

,Water_Sample_Date,Casing_Depth,Well_Depth,Static_Water_Depth,Flow_Rate,pH,Geology,Soil_Type,Landuse,Longitude,Latitude,Stream_Distance,Elevation,Detect_Concentration
,<date>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
1,2012-09-24,52,165,41,60.0,7.7,CZmd3,BdB2,81,-80.29918,35.17970,811.9613,611.499,0
2,2015-12-17,40,445,42,2.0,7.3,CZfv2,GsB,81,-80.31061,35.15487,341.7654,688.452,0
3,2015-02-02,45,160,40,40.0,7.4,CZmd3,GsB,81,-80.33170,35.16158,634.2669,692.789,0
4,2012-10-22,42,440,57,1.5,8.0,CZmd2,BaB,42,-80.32263,35.13962,855.4713,692.789,1
5,2011-01-03,48,120,42,25.0,7.1,CZmd2,BdB2,43,-80.32911,35.13027,163.5688,683.387,0
6,2015-12-15,60,280,32,10.0,8.2,CZmd2,BaB,22,-80.32205,35.10479,725.7672,614.599,1


# Random Forest
- an ensemble learning method operating by constructing a multitude of decision trees at training time, which uses multiple methods to obtain a better predictive performance and includes bagging and random forest
- algorithm uses a bootstrop aggregation, to reduce overfitting the training datset but only a subset of the features are used hence decorrelation of predictors

In [10]:
rf_classification = function(dataset, outcome, pred_outcome){
    # setting for reproducibility
    set.seed(12)
    # splitting data into training and testing sets
    dataset_index = createFolds(dataset[[outcome]], k = 5) # 5 fold CV
    
    ntree_values = c(50, 250, 500) # number of trees 
    p = dim(dataset)[2] - 1 # number of variables in dataset
    mtry_values = c(sqrt(p), p/2, p/3) # number of predictors

    metrics = data.frame()
    variable_importance_df = data.frame()
    roc_objects = c()
    threshold_data = data.frame()
    
    for (i in 1:length(dataset_index)){
        
        data_train = dataset[-dataset_index[[i]],]
        # using SMOTE to balance the class distribution
        balanced_data_train = smotenc(data_train, outcome)
        data_test = dataset[dataset_index[[i]],]

        # will use ntree and mtry values to determine which combination yields the smallest MSE
        reg_rf_pred_tune = list()
        rf_OOB_errors = list()
        rf_error_df = data.frame()
        for (j in 1:length(ntree_values)){
            for (k in 1:length(mtry_values)){
                reg_rf_pred_tune[[k]] = randomForest(as.formula(paste0(outcome, "~.")), data = balanced_data_train, 
                                                     ntree = ntree_values[j], mtry = mtry_values[k])
                rf_OOB_errors[[k]] = data.frame("Tree Number" = ntree_values[j], "Variable Number" = mtry_values[k], 
                                       "OOB_errors" = reg_rf_pred_tune[[k]]$err.rate[ntree_values[j],1])
                rf_error_df = rbind(rf_error_df, rf_OOB_errors[[k]])
            }
        }

        # finding the lowest OOB error using best number of predictors at split and refitting OG tree
        best_oob_errors <- which(rf_error_df$OOB_errors == min(rf_error_df$OOB_errors))

        reg_rf <- randomForest(as.formula(paste0(outcome, "~.")), data = balanced_data_train,
                               ntree = rf_error_df$Tree.Number[min(best_oob_errors)],
                               mtry = rf_error_df$Variable.Number[min(best_oob_errors)])

        # predicting on test set
        data_test[[pred_outcome]] = predict(reg_rf, newdata = data_test, type = "response")
        
        matrix = confusionMatrix(data = data_test[[pred_outcome]], reference = data_test[[outcome]], 
                                     positive = "1")

        # calculating AUC
        auc = auc(response = data_test[[outcome]], predictor = factor(data_test[[pred_outcome]], ordered = TRUE))
        
        # calculating values to plot ROC curve later
        roc_obj = roc(response = data_test[[outcome]], predictor = factor(data_test[[pred_outcome]], ordered = TRUE))

        # Return max Youden's index, with specificity and sensitivity
        best_thres_data = data.frame(coords(roc_obj, x = "best", best.method = c("youden", "closest.topleft")))
        threshold_data = rbind(threshold_data, best_thres_data)
        
        # extracting accuracy, sens, spec, PPV to take mean later
        matrix_values = data.frame(t(c(matrix$byClass[11])), t(c(matrix$byClass[1:3])), auc)
        
        # extracting variable importance
        var_importance_values = data.frame(importance(reg_rf)) %>%
            rownames_to_column(var = "Predictor")
        variable_importance_df = rbind(variable_importance_df, var_importance_values)
   
        # adding values to df
        metrics = rbind(metrics, matrix_values)
        
    }
    
    # taking averages/sd 
    metrics = metrics %>%
        summarise(`Balanced Accuracy` = mean(Balanced.Accuracy), Sensitivity = mean(Sensitivity), 
              Specificity = mean(Specificity), PPV = mean(Pos.Pred.Value), AUC = mean(auc))
    
    variable_importance_df = variable_importance_df %>%
        group_by(Predictor) %>%
        summarise(MeanDecreaseGini = mean(MeanDecreaseGini)) %>%
        # sorting by most important variables
        arrange(-MeanDecreaseGini)
  
    # return training set, matrix, variable importance values, (last) roc object, best threshold data
    return(list(balanced_data_train, metrics, variable_importance_df, roc_obj, threshold_data))

}

In [11]:
# calling fn
rf_values_arsenic_detect = rf_classification(arsenic_detect_df[,c(1:6,10:14)], "Detect_Concentration", 
                                             "pred_Detect_Concentration")
rf_values_arsenic_five = rf_classification(arsenic_five_df[,c(1:6,10:14)], "Five_Threshold", 
                                             "pred_Five_Threshold")
rf_values_arsenic_ten = rf_classification(arsenic_ten_df[,c(1:6,10:14)], "Ten_Threshold", 
                                             "pred_Ten_Threshold")

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting leve

In [12]:
# viewing results
rf_confusion_matrix = data.frame(Model = rep(c("RF Classification"), times = 3),
    Outcome = c("Detect_Concentration", "Five_Threshold", "Ten_Threshold"), Kernel = NA,
            rbind(rf_values_arsenic_detect[[2]], rf_values_arsenic_five[[2]], rf_values_arsenic_ten[[2]])) %>%
    rename(`Balanced Accuracy` = Balanced.Accuracy)

rf_confusion_matrix

# viewing most significant features
rf_values_arsenic_detect[[3]]
rf_values_arsenic_five[[3]]
rf_values_arsenic_ten[[3]]

Model,Outcome,Kernel,Balanced Accuracy,Sensitivity,Specificity,PPV,AUC
<chr>,<chr>,<lgl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
RF Classification,Detect_Concentration,NA,0.7791275,0.6864103,0.8718447,0.6836594,0.7791275
RF Classification,Five_Threshold,NA,0.7615969,0.6599190,0.8632748,0.6513291,0.7615969
RF Classification,Ten_Threshold,NA,0.7468047,0.5985755,0.8950339,0.5656666,0.7468047


Predictor,MeanDecreaseGini
<chr>,<dbl>
Longitude,121.84630
pH,51.75217
Latitude,45.55831
Casing_Depth,40.60509
Well_Depth,30.14297
Flow_Rate,28.53085
Elevation,24.49825
Water_Sample_Date,23.36812
Stream_Distance,22.80941


Predictor,MeanDecreaseGini
<chr>,<dbl>
Longitude,121.99304
pH,55.20727
Latitude,46.81851
Casing_Depth,37.46291
Flow_Rate,31.37815
Well_Depth,30.15409
Elevation,24.23227
Stream_Distance,23.12854
Water_Sample_Date,22.53528


Predictor,MeanDecreaseGini
<chr>,<dbl>
Longitude,147.28182
pH,56.97209
Latitude,53.77734
Casing_Depth,47.48790
Flow_Rate,35.55158
Well_Depth,30.30357
Static_Water_Depth,26.03656
Elevation,22.78932
Stream_Distance,22.70726


Rerunning RF model with noise variables to determine, which predictors are significant above the background noise.

In [13]:
train_vars_noise_arsenic_detect = rf_values_arsenic_detect[[1]]
train_vars_noise_arsenic_five = rf_values_arsenic_five[[1]]
train_vars_noise_arsenic_ten = rf_values_arsenic_ten[[1]]

noise_df = function(train_vars_noise){
    set.seed(8)
    # Add random noise predictors as an additional method to evaluate model performance
    # Adding a column that contains randomly shuffled values from one of the molecules; sampling with replacement
    train_vars_noise$noise1 = sample(train_vars_noise[[colnames(train_vars_noise[10])]], replace = TRUE) 
    train_vars_noise$noise2 = sample(train_vars_noise[[colnames(train_vars_noise[2])]], replace = TRUE)
    train_vars_noise$noise3 = sample(train_vars_noise[[colnames(train_vars_noise[5])]], replace = TRUE)
    train_vars_noise$noise4 = sample(train_vars_noise[[colnames(train_vars_noise[6])]], replace = TRUE)
    train_vars_noise$noise5 = sample(train_vars_noise[[colnames(train_vars_noise[9])]], replace = TRUE)
    
    return(train_vars_noise)
}

# calling fn
noise_training_rf_dataset_arsenic_detect = noise_df(train_vars_noise_arsenic_detect)
noise_training_rf_dataset_arsenic_five = noise_df(train_vars_noise_arsenic_five)
noise_training_rf_dataset_arsenic_ten = noise_df(train_vars_noise_arsenic_ten)
head(noise_training_rf_dataset_arsenic_detect)

,Water_Sample_Date,Casing_Depth,Well_Depth,Static_Water_Depth,Flow_Rate,pH,Longitude,Latitude,Stream_Distance,Elevation,Detect_Concentration,noise1,noise2,noise3,noise4,noise5
,<date>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,2015-12-17,40,445,42,2.0,7.3,-80.31061,35.15487,341.7654,688.452,0,526.7510,80.00000,1.732452,8.000000,1010.7342
2,2015-02-02,45,160,40,40.0,7.4,-80.33170,35.16158,634.2669,692.789,0,635.0540,44.24378,12.000000,7.100000,1665.8664
3,2012-10-22,42,440,57,1.5,8.0,-80.32263,35.13962,855.4713,692.789,1,601.3668,39.87750,18.000000,7.171503,812.8210
4,2011-01-03,48,120,42,25.0,7.1,-80.32911,35.13027,163.5688,683.387,0,611.5912,48.42387,1.500000,7.900000,268.4793
5,2015-12-15,60,280,32,10.0,8.2,-80.32205,35.10479,725.7672,614.599,1,609.5320,45.00000,5.795769,8.000000,601.0162
6,2008-10-22,60,440,40,15.0,7.7,-80.31621,35.06067,646.9204,614.599,1,659.5289,47.00000,24.000000,8.100000,745.0010


In [14]:
# calling fn
noise_rf_values_arsenic_detect = rf_classification(noise_training_rf_dataset_arsenic_detect, "Detect_Concentration", 
                                             "pred_Detect_Concentration")
noise_rf_values_arsenic_five = rf_classification(noise_training_rf_dataset_arsenic_five, "Five_Threshold", 
                                             "pred_Five_Threshold")
noise_rf_values_arsenic_ten = rf_classification(noise_training_rf_dataset_arsenic_ten, "Ten_Threshold", 
                                             "pred_Ten_Threshold")

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting leve

In [15]:
# viewing results
noise_rf_confusion_matrix = data.frame(Model = rep(c("RF w/ Noise"), times = 3),
    Outcome = c("Detect_Concentration", "Five_Threshold", "Ten_Threshold"),
            rbind(noise_rf_values_arsenic_detect[[2]], noise_rf_values_arsenic_five[[2]], 
                  noise_rf_values_arsenic_ten[[2]]))

noise_rf_confusion_matrix

Model,Outcome,Balanced.Accuracy,Sensitivity,Specificity,PPV,AUC
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
RF w/ Noise,Detect_Concentration,0.8494270,0.8662944,0.8325595,0.8391344,0.8494270
RF w/ Noise,Five_Threshold,0.8385542,0.8578313,0.8192771,0.8297150,0.8385542
RF w/ Noise,Ten_Threshold,0.8946237,0.9397849,0.8494624,0.8626882,0.8946237


Calculating the average noise rank to determine, which predictors ranked higher than the highest noise rank.

In [16]:
noise_importance_values_detect = noise_rf_values_arsenic_detect[[3]] %>%
    column_to_rownames(var = "Predictor")
noise_importance_values_five = noise_rf_values_arsenic_five[[3]] %>%
    column_to_rownames(var = "Predictor")
noise_importance_values_ten = noise_rf_values_arsenic_ten[[3]] %>%
    column_to_rownames(var = "Predictor")

In [17]:
highest_noise_rank = function(noise_importance_values){
    # this gives us values between 0 and 1
    # the closer the number gets to 1 the lower it ranks, so the number closest to 1 is the max rank
    noise1rank = which(rownames(noise_importance_values) == "noise1")/nrow(noise_importance_values)
    noise2rank = which(rownames(noise_importance_values) == "noise2")/nrow(noise_importance_values)
    noise3rank = which(rownames(noise_importance_values) == "noise3")/nrow(noise_importance_values)
    noise4rank = which(rownames(noise_importance_values) == "noise4")/nrow(noise_importance_values)
    noise5rank = which(rownames(noise_importance_values) == "noise5")/nrow(noise_importance_values)

    # figuring out which is the highest mean decrease gini or background noise importance value
    highest_noise_rank = min(noise1rank, noise2rank, noise3rank, noise4rank, noise5rank)
    
    return(highest_noise_rank)
}

# calling fn
highest_noise_rank_detect = highest_noise_rank(noise_importance_values_detect)
highest_noise_rank_five = highest_noise_rank(noise_importance_values_five)
highest_noise_rank_ten = highest_noise_rank(noise_importance_values_ten)
highest_noise_rank_detect
highest_noise_rank_five
highest_noise_rank_ten

[1] 0.7333333

[1] 0.7333333

[1] 0.7333333

In [18]:
last_significant_rank = function(noise_importance_values, highest_noise_rank){
    for (i in 1:length(noise_importance_values$MeanDecreaseGini)){
    # if the row number (index)/# of variables is less than the highest noise rank the loop will continue
        if (i/length(noise_importance_values$MeanDecreaseGini) > highest_noise_rank){
            last_significant_var = i - 1
            break
        }
    }    
    return(last_significant_var)
}

# calling fn
last_significant_rank_detect = last_significant_rank(noise_importance_values_detect, highest_noise_rank_detect)
last_significant_rank_five = last_significant_rank(noise_importance_values_five, highest_noise_rank_five)
last_significant_rank_ten = last_significant_rank(noise_importance_values_ten, highest_noise_rank_ten)
last_significant_rank_detect
last_significant_rank_five
last_significant_rank_ten

[1] 11

[1] 11

[1] 11

In [19]:
# now getting those significant predictors
significant_predictors_detect = rownames(noise_importance_values_detect)[1:last_significant_rank_detect]
significant_predictors_five = rownames(noise_importance_values_five)[1:last_significant_rank_five]
significant_predictors_ten = rownames(noise_importance_values_ten)[1:last_significant_rank_ten]

In [20]:
# converting row names back to columns to make next step easier
noise_importance_values_detect = noise_importance_values_detect %>%
    rownames_to_column(var = "Predictor")
noise_importance_values_five = noise_importance_values_five %>%
    rownames_to_column(var = "Predictor")
noise_importance_values_ten = noise_importance_values_ten %>%
    rownames_to_column(var = "Predictor")

In [21]:
# getting dataframe ready to export variable importance ranks from above and whether those features passed the 
# background filter
noise_variable_importance_ranks = function(noise_importance_values_df, significant_predictors, outcome, predictors){
    # ADD WORDS!!!!!!!
    
    # filtering the original rank df for predictors that were significant (above random noise)
    filtered_significant_ranks_df = noise_importance_values_df %>%
        filter(Predictor %in% significant_predictors) %>%
        arrange(-MeanDecreaseGini)
    
    # adding a column denoting if the feature was above random noise
    filtered_significant_ranks_df$Passed_Filter = c(rep(c("Yes"), 
                                                        times = length(filtered_significant_ranks_df$Predictor)))
    
    final_df = full_join(filtered_significant_ranks_df, noise_importance_values_df)
    
    # adding a col for outcome and the predictors used
    final_df$Outcome = outcome
    final_df$Predictors = predictors
    return(final_df)
}

# calling fn
significant_predictors_df_detect = noise_variable_importance_ranks(noise_importance_values_detect, 
                                                                   significant_predictors_detect, "Detect", "No Geology, Soil Type, Land Use")
significant_predictors_df_five = noise_variable_importance_ranks(noise_importance_values_five, 
                                                                 significant_predictors_five, "Five", "No Geology, Soil Type, Land Use")
significant_predictors_df_ten = noise_variable_importance_ranks(noise_importance_values_ten, 
                                                                significant_predictors_ten, "Ten", "No Geology, Soil Type, Land Use")

# creating 1 df
significant_predictors_df = rbind(significant_predictors_df_detect, significant_predictors_df_five,
                                 significant_predictors_df_ten)
head(significant_predictors_df)

Joining with `by = join_by(Predictor, MeanDecreaseGini)`
Joining with `by = join_by(Predictor, MeanDecreaseGini)`
Joining with `by = join_by(Predictor, MeanDecreaseGini)`


,Predictor,MeanDecreaseGini,Passed_Filter,Outcome,Predictors
,<chr>,<dbl>,<chr>,<chr>,<chr>
1,Longitude,96.72564,Yes,Detect,"No Geology, Soil Type, Land Use"
2,pH,33.54635,Yes,Detect,"No Geology, Soil Type, Land Use"
3,Casing_Depth,31.44684,Yes,Detect,"No Geology, Soil Type, Land Use"
4,Latitude,28.98030,Yes,Detect,"No Geology, Soil Type, Land Use"
5,Flow_Rate,23.28612,Yes,Detect,"No Geology, Soil Type, Land Use"
6,Well_Depth,17.16448,Yes,Detect,"No Geology, Soil Type, Land Use"


# Support Vector Machine (SVM)
- supervised learning models that can predict continuous (regression) or grouped/dichotomous (classification) outcomes
- predicts by projecting them onto a high dimensional space and uses kernels to make the data more separable (unfortunately makes interpretability of model results more difficult)
- does a better job at handling a large number of predictors since p > n
- Compared to other classification algorithms, this approach can reliably classify chemicals while avoiding overfitting and reducing susceptibility to noisy or meaningless data

In [22]:
#model errors are calculated to assess the predictive accuracy of the model
svm_classification = function(dataset, model, outcome, pred_outcome, gamma_values, cost_values, elsilon_values){
  
    #setting seed for reproducibility
    set.seed(12)
    
    #splitting data into training and testing sets
    dataset_index = createFolds(dataset[[outcome]], k = 5) #5 fold CV
    metrics = data.frame()
    for (i in 1:length(dataset_index)){
        
        data_train = dataset[-dataset_index[[i]],]
        # using SMOTE to balance the class distribution
        balanced_data_train = smotenc(data_train, outcome)
        data_test = dataset[dataset_index[[i]],]

        #now pruning parameters (based on the training dataset to prevent overfitting)
        svr_tune <- tune(svm, as.formula(paste0(outcome, "~.")), data = balanced_data_train, kernel = model, 
                       ranges = list(elsilon = elsilon_values, cost = cost_values, gamma = gamma_values)) 

        #choosing best model
        best_svm <- svr_tune$best.model

        #predicting with tuned parameters 
        data_test[[pred_outcome]] <- predict(best_svm, newdata = data_test, type = "response")

        matrix = confusionMatrix(data = data_test[[pred_outcome]], reference = data_test[[outcome]], 
                                 positive = "1")
 
        #calculating AUC
        auc = auc(response = data_test[[outcome]], predictor = factor(data_test[[pred_outcome]], ordered = TRUE))
        #extracting accuracy, sens, spec, PPV to take mean later
        matrix_values = data.frame(t(c(matrix$byClass[11])), t(c(matrix$byClass[1:3])), auc)
   
        #adding values to df
        metrics = rbind(metrics, matrix_values)
  }
  
  #taking averages/sd 
  metrics = metrics %>%
    summarise(`Balanced Accuracy` = mean(Balanced.Accuracy), Sensitivity = mean(Sensitivity), 
              Specificity = mean(Specificity), PPV = mean(Pos.Pred.Value), AUC = mean(auc)) %>%
    mutate(Model = "SVM", Outcome = outcome, Kernel = model) 

  metrics = metrics[,c(6:8,1:5)]
  
  return(metrics)
}

In [23]:
#calling fn
svm_detect_linear = svm_classification(arsenic_detect_df[,c(1:6,10:14)], "linear", "Detect_Concentration", "pred_Detect_Concentration",
                                            0.035, 1:5, seq(0,1,0.2))
svm_detect_radial = svm_classification(arsenic_detect_df[,c(1:6,10:14)], "radial", "Detect_Concentration", "pred_Detect_Concentration",
                                            0.035, 1:5, seq(0,1,0.2))
svm_detect_polynomial = svm_classification(arsenic_detect_df[,c(1:6,10:14)], "polynomial", "Detect_Concentration", "pred_Detect_Concentration",
                                            0.035, 1:5, seq(0,1,0.2))
svm_five_linear = svm_classification(arsenic_five_df[,c(1:6,10:14)], "linear", "Five_Threshold", "pred_Five_Threshold",
                                            0.035, 1:5, seq(0,1,0.2))
svm_five_radial = svm_classification(arsenic_five_df[,c(1:6,10:14)], "radial", "Five_Threshold", "pred_Five_Threshold",
                                            0.035, 1:5, seq(0,1,0.2))
svm_five_polynomial = svm_classification(arsenic_five_df[,c(1:6,10:14)], "polynomial", "Five_Threshold", "pred_Five_Threshold",
                                            0.035, 1:5, seq(0,1,0.2))
svm_ten_linear = svm_classification(arsenic_ten_df[,c(1:6,10:14)], "linear", "Ten_Threshold", "pred_Ten_Threshold",
                                            0.035, 1:5, seq(0,1,0.2))
svm_ten_radial = svm_classification(arsenic_ten_df[,c(1:6,10:14)], "radial", "Ten_Threshold", "pred_Ten_Threshold",
                                            0.035, 1:5, seq(0,1,0.2))
svm_ten_polynomial = svm_classification(arsenic_ten_df[,c(1:6,10:14)], "polynomial", "Ten_Threshold", "pred_Ten_Threshold",
                                            0.035, 1:5, seq(0,1,0.2))

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting leve

In [24]:
# creating final df
final_df = rbind(rf_confusion_matrix,
                 svm_detect_linear, svm_detect_radial, svm_detect_polynomial, svm_five_linear, svm_five_radial,
               svm_five_polynomial, svm_ten_linear, svm_ten_radial, svm_ten_polynomial)

final_df

Model,Outcome,Kernel,Balanced Accuracy,Sensitivity,Specificity,PPV,AUC
<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
RF Classification,Detect_Concentration,NA,0.7791275,0.6864103,0.8718447,0.6836594,0.7791275
RF Classification,Five_Threshold,NA,0.7615969,0.6599190,0.8632748,0.6513291,0.7615969
RF Classification,Ten_Threshold,NA,0.7468047,0.5985755,0.8950339,0.5656666,0.7468047
SVM,Detect_Concentration,linear,0.7755209,0.7626923,0.7883495,0.5857145,0.7755209
SVM,Detect_Concentration,radial,0.7694791,0.7273077,0.8116505,0.6056675,0.7694791
SVM,Detect_Concentration,polynomial,0.7600180,0.8229487,0.6970874,0.5114274,0.7600180
SVM,Five_Threshold,linear,0.7647668,0.7472335,0.7823002,0.5617370,0.7647668
SVM,Five_Threshold,radial,0.7432157,0.7060729,0.7803585,0.5452377,0.7432157
SVM,Five_Threshold,polynomial,0.7644879,0.8449393,0.6840366,0.5006961,0.7644879


In [25]:
# exporting
write.xlsx(rf_confusion_matrix, paste0(Output,"/", "As_Prediction_Confusion_Matrix_", cur_date, ".xlsx"), 
           rowNames = FALSE)
write.xlsx(significant_predictors_df, paste0(Output,"/", "Variable_Importance_", cur_date, ".xlsx"), rowNames = FALSE)